In [15]:
import csv, io
import json
from bs4 import BeautifulSoup
import codecs, prettytable
import requests
import codecs
from selenium.webdriver import Chrome


header_data={
		"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
		"Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
		"referer" : "https://forum.gamer.com.tw/B.php?bsn=60559&subbsn=7"
		}

In [16]:
# 取得所有標題的 snA

# --------subbsn----------
Apple = "7" 
p_Apple = 18

Sony = "13" 
p_Sony = 9

Samsung = "12" 
p_Samsung = 10

XiaoMi = "11" 
p_Mi = 6

Google = "22" 
p_Google = 2

# 需設定參數
classNow = Apple 
page_count = p_Apple 
def get_snA(page_count, classNow):
    snA_list = []
    for page in range(1, page_count+1):
        r1 = requests.get(
                f"https://forum.gamer.com.tw/B.php?bsn=60559&subbsn={classNow}",
                headers=header_data,
                params={
                    "page":str(page)
                    }
            )
        b1=BeautifulSoup(r1.text, "html.parser")
        a1 = b1.find_all("td",{"class":"b-list__summary"})


        for a2 in a1:
            snA = a2.find("a")
            # print(snA.attrs["name"])
            snA_list.append(snA.attrs["name"])
            
    snA_list.remove("48476")
        
    return snA_list

snA_list = get_snA(page_count, classNow)
print(snA_list)
print(len(snA_list))



['50083', '50073', '50076', '50066', '50064', '50038', '50021', '50023', '49991', '49985', '49962', '49938', '49906', '49898', '49896', '49894', '49886', '49874', '49857', '49698', '49819', '49795', '49788', '49785', '49771', '49721', '49724', '49702', '49526', '49688', '49656', '49679', '49661', '49652', '49640', '49600', '49581', '49539', '49512', '49493', '49468', '49407', '49398', '49394', '49373', '49287', '49271', '49264', '49262', '49253', '49084', '49221', '49176', '49209', '49186', '49120', '49117', '49087', '49102', '49094', '49063', '49059', '48971', '48884', '48860', '48851', '48823', '48794', '48765', '48777', '48770', '48768', '48759', '48734', '48711', '48699', '48634', '48598', '48613', '48583', '48590', '48571', '48503', '48504', '48485', '48484', '48479', '48469', '48438', '48432', '48404', '48365', '48320', '48296', '48297', '48182', '48166', '48164', '48087', '48077', '46876', '42799', '47418', '46126', '47356', '47224', '47167', '47131', '46629', '46972', '46839', 

In [17]:
snA_list=[50083]
# snA_list.append("49698")
# print(snA_list)
# snA_list.index("42749")
# a = snA_list.index("42749")
# snA_list = snA_list[a:]
# snA_list

In [18]:
def get_floor(bs):
    floor_list=[]
    floor = bs.find_all("p",{"class":"BH-pagebtnA"})
    
    for f in floor:
#         print(f.text[-1])
        floor_list.append(f.text[-1])
#   print(floor_list)
    return floor_list

In [19]:
def get_page_parser(snA, classNow):
    r = requests.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&subbsn={classNow}",
                     headers=header_data,
                     params={
                     }
                     )
    r.encoding = "utf-8"
    bs = BeautifulSoup(r.text, "html.parser")
    return bs

In [20]:
def get_commentId(snA, floor, classNow):
    r2 = requests.get(
                f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={floor}&subbsn={classNow}",
                headers=header_data,
                params={
                    }
            )
#   print(range(1, int(page_list[0]) + 1))
    print("正在處理第幾頁:", floor)
    commentId_list=[]
    b2=BeautifulSoup(r2.text, "html.parser")
    a1 = b2.find_all("textarea")
    for a in a1:
#             print(a.attrs["data-snb"])
        commentId_list.append(a.attrs["data-snb"])
    print(commentId_list)
    return commentId_list


In [25]:
import time

# 取得該snA展開留言編號
with open("V3test.csv", "a", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["標題", "內容", "使用者評論"])

for snA in snA_list:
    
    data = []
    
    bs = get_page_parser(snA, classNow)
    floor_list = get_floor(bs)

    driver = Chrome("./chromedriver")
    for floor in range(1, int(floor_list[0]) + 1):
        
        commentId_list = get_commentId(snA, floor, classNow)
        # 打開網址
        driver.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={floor}&subbsn={classNow}")
        # find -> find-element
        # find all -> find_elements
        for id_ in commentId_list:      
            try:
                time.sleep(3)
                driver.find_element_by_id("showoldCommend_"+ id_).click()
                time.sleep(3)
            except:
                continue   
#     -------------------------------------------------------------------------------------            
        comment = driver.find_elements_by_class_name("reply-content__article")
        content = driver.find_elements_by_class_name("c-article__content")
        title = driver.find_elements_by_class_name("c-post__header__title ")
        title_list = []
        content_list = []
        comment_list = []
        result = dict()

        for til in title:
            title_list.append(til.text)
            print(til.text)
        for con in content:
            content_list.append(con.text)
            print(con.text)   
        for com in comment:
            comment_list.append(com.text)
            print(com.text) 
        
        result["title"] = title_list
        result["content"] = content_list
        result["comment"] = comment_list
        data.append(result)
        
        with open("V3test.csv", "a", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            for item in data:
                writer.writerow([item['title'], item['content'], item['comment']])
                     
    driver.close() 
        

正在處理第幾頁: 1
['150520', '150522']
【問題】從三星平板S2 跳到IPAD PRO 2020
使用了快6年的三星平板S2，被更新害慘後不勘使用，本來是沒打算入手平板
但是因為上大夜班環境可以讓我無所畏懼的使用平板，趁這次IPAD PRO新出的 等了兩天終於入手了。
11吋 128G WIFI 版本，本來是打算入手12.9吋256GWIFI 想了想 入手太大台好像沒必要
主要網頁 小說 3D遊戲 吃雞 影片
超尷尬 剛剛回覆錯地方

————

是個被我蠱惑去買 iPad 的巴友呢 (X

不過記得記好你的 Apple ID 賬號和密碼

忘了會很麻煩的
不得不說整體真的很漂亮
煎餅好ㄘ你得也不錯的呢
小心彎版喔
結月葵我婆我會注意的 已經買好外套給她穿 不然使用起來怕怕的
可以說一下為什麼沒有考慮最近三星的tab s6 lite或tab s6呢？
Scott本來是有考慮S6 考慮到介面上的習慣性吧跟一些IOS才有的應用軟體
恭喜 如常放後背包 可考慮買AC+ OuO
幻滅星旋我都放機車裡面OAO 比較不怕被擠壓到
幻滅星旋AC+ 怎麼買??
網站上刷卡就能買了或打電話或直營店。
幻滅星旋了解 謝謝你 我來去看一下
請問保護殼買哪一種的呀><
龜龜一般的皮套殼
了解！我也訂了一台><晚點去取貨
龜龜歡迎入坑
您好 先恭喜樓主入手新平板請問 是今天開賣?
有喔 我都用共同的 本身也是使用I鳳手機
ლ(́◕◞౪◟◕‵ლ)想問你用的是什麼 iPhone
樹上的Apple不能吃I7 打算換今年的I12
ლ(́◕◞౪◟◕‵ლ)iPad 多工處理可以看我之前的文章，然後基本上你熟悉了 iPad 的交互後你就會無縫接軌全螢幕 iPhone 的交互了
樹上的Apple不能吃了解 目前還在摸索 她的使用方式
樹上的Apple不能吃我會再考慮看看買 巧控鍵盤跟筆
目前只覺得資料 相片能夠雲端一起用方便OuO和網絡分享方便。
ლ(́◕◞౪◟◕‵ლ)筆可以買，巧控鍵盤別了
樹上的Apple不能吃 竟然回錯OwO
幻滅星旋好
樹上的Apple不能吃筆 我再找找看有沒有比較便宜的
樹上的Apple不能吃 筆真的是最值得購買的配件呢。
幻滅星旋筆 主要能拿來做甚麼?
ლ(́◕◞౪◟◕‵ლ) 就筆記吧，戴套子後拿來滑網頁 點遊戲也不錯。
幻滅星旋筆記比較少用 點遊戲 影片 

In [10]:
def main():
    data = []
    
    for i in range(1, 66):
        ret = get_page_detail(i)
        # data.append(ret)
        print("[", i, "]", "=====", ret)
        with open("MIUserComment.csv", "a", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["標題", "內容", "使用者評論"])
            for item in ret:
                writer.writerow([item['title'], item['content'], item['comments']])


    #    for i in range(1, 66):

我有試過itunes，但是根本沒有辦法傳任何東西
icloud又非常慢，傳個小影片電腦好了手機的icloud資料夾要很久之後才會顯示
請問有什麼好方法嗎？
推薦一個軟體 sendanywhere，我家開照相館客人傳送照片都用這個方式，而且不限於照片，影片音訊文件都可以
手機跟電腦都安裝究OK了
既然都用蘋果了，你也只能麻煩一點的慢慢傳了，要直接把手機當隨身碟使用，無解。
感謝大家的建議，我每個都試試看好了
正解:購買一隻S10
蔬菜下一隻手機是有想換回安卓了
很棒的app
